# Estimation statistics

Statistics are an essential mathematical tool in biomedical research for describing, analyzing, and interpreting experimental data. Many useful guidelines for reporting statistics in scientific journals have been published in recent years. To name just a few: [Kass et al., 2016](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1004961), [Makin and Orban de Xivry, 2019](https://elifesciences.org/articles/48175), and the [Statistics for Biologists series](https://www.nature.com/collections/qghhqm) published by Nature. These articles are valuable resources for improving our understanding of statistics and avoiding common mistakes.

The most common statistical approach in biomedicine is Null Hypothesis Significance Testing (NHST), which involves calculating the probability of observing the data—or something more extreme—if the null hypothesis is true. This probability is referred to as the p-value. Based on this value, researchers determine whether observed differences are statistically significant.

In the last few years, several authors suggest taking complementary approaches o measuring quantitative effects and reporting uncertainty. In this regard, **estimation statistics** focus more on the effect size or, in other words, the magnitude of the effect we are investigating (how much?). This is important because results can be statistically significant using NHST, but the effect size and its error can show that the difference is almost negligible, regardless of its biological interpretation (see [Gelman and Stern, 2012](https://www.tandfonline.com/doi/abs/10.1198/000313006X152649)).

This notebook adapts the code from the paper "Moving beyond P values: data analysis with estimation graphics" by [Ho et al., 2019](https://doi.org/10.1038/s41592-019-0470-3) ([PDF](https://www.researchgate.net/publication/333884529_Moving_beyond_P_values_data_analysis_with_estimation_graphics)). Ho et al. developed DABEST (‘data analysis with bootstrap-coupled estimation for calculating and plotting estimation statistics’). DABEST is an open-source library for Matlab, Python, and R, and is also available as a web application called [estimationstats](https://www.estimationstats.com/).

The authors of this paper argue that estimation methods and plots can be a better alternative to null hypothesis significance testing. In the case of plots, the figure below shows how to improve two-group data graphics from a bar plot (a) to an estimation plot (e). Estimation plots show the following:

- All data points plotted as swarm plots, allowing readers to see the size and distributions of the samples.
- Effect size: mean, median, Cohen's d, Hedge's d, or Cliff's delta.
- Precision of effect size performed through bootstrap resampling, which generally approaches a normal distribution (central limit theorem).
- Confidence intervals for effect size based on the bootstrap resampling.

![image](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41592-019-0470-3/MediaObjects/41592_2019_470_Fig1_HTML.png?as=webp)

Figure from [Ho et al., 2019](https://doi.org/10.1038/s41592-019-0470-3).

To know more about **estimation statistics**:|
- Bernard, 2024. Statistical Analysis Is Vital at eNeuro. [Link](https://pmc.ncbi.nlm.nih.gov/articles/PMC10941633/). 
- Calin-Jageman, 2018. The New Statistics for Neuroscience Majors: Thinking in Effect Sizes. [Link](https://pmc.ncbi.nlm.nih.gov/articles/PMC6057753/). 
- Calin-Jageman and Cumming, 2019. Estimation for Better Inference in Neuroscience. [Link](https://www.eneuro.org/content/6/4/ENEURO.0205-19.2019)
- Calin-Jageman, 2021. Better Inference in Neuroscience: Test Less, Estimate More. [Link](https://www.jneurosci.org/content/42/45/8427). 
- Chang and Assam, 2016. Estimation statistics should replace significance testing. [Link](https://www.nature.com/articles/nmeth.3729).
- Cumming and Calin-Jageman. Introduction to the New Statistics. [Link](https://thenewstatistics.com/itns/).
- Gardner and Altman, 1986. Confidence intervals rather than P values: estimation rather than hypothesis testing. [Link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1339793/).
- Lakens, 2013. Calculating and reporting effect sizes to facilitate cumulative science: a practical primer for t-tests and ANOVAs. [Link](https://www.frontiersin.org/articles/10.3389/fpsyg.2013.00863/full).
- Wasserstein et al., 2019. Moving to a World Beyond “p < 0.05”. [Link](https://www.tandfonline.com/doi/full/10.1080/00031305.2019.1583913)

Some authors like Daniel Lakens advocate for a more nuanced view of NHST and p-values, emphasizing their usefulness when properly applied to support informed decision-making.

To know more about **p values**:
- Cumming, 2013. Dance of p values. [Link](https://www.youtube.com/watch?v=5OL1RqHrZQ8) and [website](https://www.youtube.com/watch?v=5OL1RqHrZQ8). 
- Lakens, 2021. The Practical Alternative to the p Value Is the Correctly Used p Value. [Link](https://journals.sagepub.com/doi/10.1177/1745691620958012).
- Nuzo, 2014. Scientific method: Statistical error. [Link](https://www.nature.com/articles/506150a).
- Wasserstein and Lazar. The ASA Statement on p-Values: Context, Process, and Purpose. [Link](https://www.tandfonline.com/doi/full/10.1080/00031305.2016.1154108). 

# Example data

Cell features dataset from the Allen Brain Atlas. You can also download the 'csv' file [here](https://celltypes.brain-map.org/data). For a description of the features, download the electrophysiology and morphology [documentation](https://community.brain-map.org/t/documentation-cell-types-database/2845).

Go to `Example_files` > `estimation_stats_dabest` > `cell_types_specimen_details.csv`, download the file and paste it into the `Data_example` folder of the notebook after you create the paths. Alternatively, if you clone the repository, paste the `estimation_stats_dabest` into the new `Data_examples` directory. After that, you can run all the cells of the notebook. 

# Import the libraries

See the **[dabest](https://acclab.github.io/DABEST-python/) tutorial** for details. Note: Dabest works with both 'wide' and 'long' datasets. Read [here](https://en.wikipedia.org/wiki/Wide_and_narrow_data) the differences between the two types. When you use long datasets, you need to specify `x` and `y` columns.

**Note**: Dabest might give errors if you use the latest `matplotlib` and `python` versions. For this notebook, I used `python=3.11`. 

Versions I used for this notebook: Pandas 1.5.3, NumPy 1.23.5, Matplotlib 3.6.3, DABEST 2024.03.29

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import dabest

# Optional: Display all columns in table
pd.set_option('display.max_columns', None)

# Create the paths

In [ ]:
notebook_name = 'estimation_stats_dabest'

# Data path to where you want to locate your folders
data_path = os.getcwd()  # Default: current directory where the Jupyter notebook is located

# Change the folder names accordingly
paths = {'data': data_path,
         'raw_data':  f'{data_path}/Data_examples/{notebook_name}/',
         'processed_data': f'{data_path}/Processed_data_examples/{notebook_name}/',
         'analysis': f'{data_path}/Analysis_examples/{notebook_name}/',         
         'plots': f'{data_path}/Analysis_examples/{notebook_name}/Plots/'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Load the dataframe

In [ ]:
dataset = pd.read_csv(paths['raw_data'] + '/cell_types_specimen_details.csv')

dataset

## Filter the table

Optional step.

In [ ]:
dataset_filtered = dataset[(dataset['structure_parent__acronym'] == 'VISp'
                           ) & (dataset['donor__species'] == 'Mus musculus')]
dataset_filtered

## Summary statistics

In [ ]:
dataset_filtered[dataset_filtered['line_name'] == 'Pvalb-IRES-Cre'].describe()

# Dabest: unpaired data

I show some examples for statistical analysis. See the links above and the 'Readme' for references on statistics. 

Documentation: [dabest load](https://acclab.github.io/DABEST-python-docs/api.html#dabest.load). 

- Unpaired Student’s t-test: `paired=None`
- Paired Student’s t-test: `paired=baseline`

Defaults: `dabest.load(data, idx=None, x=None, y=None, paired=None, id_col=None, ci=95, resamples=5000, random_seed=12345, proportional=False, delta2=False, experiment=None, experiment_label=None, x1_level=None, mini_meta=False)` 

In [ ]:
# Set the parameters for the comparisons
feature = 'nr__number_bifurcations'
groups = 'line_name'
group_names = ('Vip-IRES-Cre', 'Pvalb-IRES-Cre')

# Load the dataset
unpaired_dabest = dabest.load(dataset_filtered, y=feature, x=groups, 
                              idx=group_names,
                              paired=None)

## Mean differences: tests

For unpaired comparisons, the p-values and test statistics of Welch’s t test, Student’s t test, and Mann-Whitney U test can be used. For paired comparisons, the p-values and test statistics of the paired Student’s t and Wilcoxon tests are presented.

Note: bca (bias-corrected and accelerated confidence interval)

In [ ]:
unpaired_dabest.mean_diff

In [ ]:
stats_results = unpaired_dabest.mean_diff.statistical_tests

# Save the results
stats_results.to_csv(paths['analysis'] + feature + '_mean_diff_stats.csv')

stats_results

## Mean differences: plot

More about plotting [here](https://acclab.github.io/DABEST-python/API/plotter.html).

In [ ]:
# Gardner-Altman estimation plot
mean_plot = unpaired_dabest.mean_diff.plot(color_col="structure__layer",
                                          fig_size=(7,5))

mean_plot.savefig(paths['plots'] + feature + '_mean_diff_plot.png')

In [ ]:
# Cumming estimation plot (bootstrap effect sizes below)
mean_plot = unpaired_dabest.mean_diff.plot(float_contrast=False,
                                          raw_marker_size=5)

## Standarized effect size: tests

Standarized effect sizes were created to compare effect sizes with different scales. One standarized effect size is **Cohen d**. Cohen d is the effect size divided by the standard deviation. Thus, it is similar to a z score. 

In [ ]:
unpaired_dabest.cohens_d.statistical_tests

## Standarized effect size: plot

In [ ]:
cohen_plot = unpaired_dabest.cohens_d.plot(fig_size=(7,5)) 
cohen_plot.savefig(paths['plots'] + feature + '_cohen_plot.png')

# Dabest: repeated measures

- Multi-two-group estimation plot. Past a nested tuple `(a,b), (c, d)` to `idx`
- Shared control plot. Default when there are more than 2 columns in `idx`

In [ ]:
# Set the feature to compare groups
feature = 'nr__number_bifurcations'
neuron_type = 'Pvalb-IRES-Cre'

dataset_filtered_neuron = dataset_filtered[(dataset_filtered['line_name'] == neuron_type)]

# Load the dataset
repeated_measures_dabest = dabest.load(dataset_filtered_neuron, y=feature, x='structure__acronym', 
                                       idx=('VISp2/3', 'VISp4', 'VISp5', 'VISp6a'),
                                       id_col='id',
                                       paired=None)

In [ ]:
stats_results = repeated_measures_dabest.mean_diff.statistical_tests

stats_results.to_csv(paths['analysis'] + feature + '_repeated_measures_stats.csv')

stats_results

In [ ]:
# Change the plot visualization using the function parameters
repeated_measures_plot = repeated_measures_dabest.mean_diff.plot(custom_palette="Set2", 
                                                       raw_marker_size=4,  # data marker size
                                                       # es_marker_size=8,  # effect plot marker size
                                                       # swarm_ylim=(0, 40),  # data plot y-limits
                                                       # contrast_ylim=(-10, 10),  # effect plot y-limits
                                                      )

repeated_measures_plot.savefig(paths['plots'] + feature + '_repeated_measures_plot.png')

# Dabest: delta-delta

See [here](https://acclab.github.io/DABEST-python/) how to evaluate the effects of two interacting independent variables on a dependent variable. For example, the efficacy of a compound on wild-type and mutant animals.